In [1]:
# Install required packages and download test data
%run ./setup.py

Installing requirements
Solving environment: ...working... done

# All requested packages already installed.



Training Data: 100%|█████████▉| 1.81G/1.81G [02:44<00:00, 11.0MB/s] 


Extracting data
Done


In [2]:
# Load required packages
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import my_alg
%matplotlib

2023-01-25 16:04:00.425191: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using matplotlib backend: <object object at 0x7f5546bea250>


In [3]:
# define loss function
def ave_cross_en( y_true, y_pred):
    loss = ( keras.losses.kullback_leibler_divergence( y_true, y_pred) +  keras.losses.kld( 1 - y_true, 1- y_pred))
    return loss

In [7]:
# load dataset
mnist_x = np.load('training_data/ref_optrode_plant_128_allnorm_ccm1_128.npy')
mnist_b = np.load('training_data/ref_optrode_plant_128_allnorm_ccm1_128.npy')


In [8]:
# check shape of dataset
mnist_x.shape, mnist_b.shape

((26088, 128, 128), (26088, 128, 128))

In [9]:
# expand dimension of dataset to fit model
mnist_b = np.expand_dims( mnist_b, axis=3)
mnist_x = np.expand_dims( mnist_x, axis=3)

In [10]:
mnist_x.shape, mnist_b.shape

((26088, 128, 128, 1), (26088, 128, 128, 1))

In [11]:
# define random function to seprate dataset into train data, eval data and test data, randomly
np.random.seed(47)
arg_random = np.random.permutation( len( mnist_b))

In [12]:
# seprate dataset into train data, eval data and test data, randomly
NUM_TRAIN = int( 0.9 * len( mnist_x))
x_train = mnist_x[ arg_random[:NUM_TRAIN-1000]]
b_train = mnist_b[arg_random[:NUM_TRAIN-1000]]

x_eval = mnist_x[ arg_random[NUM_TRAIN-1000:NUM_TRAIN]]
b_eval = mnist_b[ arg_random[NUM_TRAIN-1000:NUM_TRAIN]]

x_test = mnist_x[ arg_random[NUM_TRAIN:]]
b_test = mnist_b[ arg_random[NUM_TRAIN:]]


In [13]:
# define model and compile it
model = my_alg.build_model_128()
model.compile(optimizer=keras.optimizers.Adam(1e-3, amsgrad= True), loss=ave_cross_en, metrics=['mae'])

2023-01-25 16:08:17.325443: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# train model: set batch_size and epochs
model.fit(b_train, x_train, batch_size=16, epochs=17, verbose=1,validation_data= ( b_eval, x_eval))

Epoch 1/17


2023-01-25 16:08:27.759393: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1473183744 exceeds 10% of free system memory.
2023-01-25 16:08:28.201518: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1473183744 exceeds 10% of free system memory.


   7/1405 [..............................] - ETA: 41:25 - loss: 0.2711 - mae: 0.2134


KeyboardInterrupt



In [ ]:
# predict train data
train_pred = model.predict( b_train)

In [ ]:
train_fig = my_alg.show_some_results( b_train, x_train, train_pred,20)

In [ ]:
# predict test data
test_pred = model.predict( b_test)

In [ ]:
test_fig = my_alg.show_some_results( b_test, x_test, test_pred,20, img_size=5)